In [4]:
import nbformat

# Read your notebook (assuming version 4 for example purposes)
nb = nbformat.read("ml_project.ipynb", as_version=4)

# Normalize the notebook to add missing id fields and other updates
nbformat.validator.validate(nb)

# Write the normalized notebook back to a file
nbformat.write(nb, "ml_project_normalized.ipynb")

In [5]:
%run ml_project_normalized.ipynb

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
dropped columns
cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned color
cleaned breed
dropped columns


/tmp/ipykernel_974738/1199118304.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt_series = pd.to_datetime(df['intake_time'], errors='coerce')


cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned color
cleaned breed
Done running ml_project.ipynb.


In [6]:
#kog_reg
nb = nbformat.read("log_reg.ipynb", as_version=4)
nbformat.validator.validate(nb)
nbformat.write(nb, "log_reg_normalized.ipynb")

# Random Frest
nb = nbformat.read("random_forest.ipynb", as_version=4)
nbformat.validator.validate(nb)
nbformat.write(nb, "random_forest_normalized.ipynb")

# X Random Trees
nb = nbformat.read("x_random_trees.ipynb", as_version=4)
nbformat.validator.validate(nb)
nbformat.write(nb, "x_random_trees_normalized.ipynb")

In [7]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import balanced_accuracy_score
from imblearn.pipeline import Pipeline as ImbPipeline  # Important for SMOTE compatibility
from imblearn.over_sampling import SMOTENC

In [8]:
# rerun when i make changes
#%run log_reg.ipynb
#%run random_forest.ipynb
#%run x_random_trees.ipynb
df_train.head()

,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,intake_year,intake_month,intake_hour,primary_color,is_mix,size,outcome_type
0,Stray,Normal / Behavior,Dog,Spayed Female,96.0,2,2015,7,12,white,0,3,Return to Owner
1,Stray,Normal / Behavior,Dog,Intact Male,11.0,7,2016,4,18,sable,1,2,Return to Owner
2,Public Assist,Normal / Behavior,Cat,Neutered Male,24.0,Common,2022,5,0,orange,0,2,Transfer
3,Owner Surrender,Normal / Behavior,Dog,Neutered Male,24.0,2,2017,2,12,chocolate,1,4,Return to Owner
4,Public Assist,Normal / Behavior,Dog,Neutered Male,72.0,3,2019,4,9,black,1,5,Return to Owner


In [9]:
def train_voting_classifier(X_train, y_train, X_test, models, voting='soft'):
    """
    Trains a VotingClassifier with the given models and evaluates it on the test data.
    
    Parameters:
    - X_train: Training features
    - y_train: Training labels
    - X_test: Test features
    - models: List of tuples (model_name, model_instance)
    - voting: 'hard' or 'soft' for the type of voting to be used (default is 'soft')
    
    Returns:
    - trained_model: Trained VotingClassifier
    - test_predictions: Predictions on the test set
    """
    # Initialize the VotingClassifier with the given models
    voting_clf = VotingClassifier(estimators=models, voting=voting, n_jobs=-1)
    
    # Fit the model
    voting_clf.fit(X_train, y_train) #   voting_clf.fit_transform(X_test,)
    
    # Make predictions
    test_predictions = voting_clf.predict(X_test)
    
    # Evaluate the model
    accuracy = balanced_accuracy_score(y_train, voting_clf.predict(X_train))
    print(f"Training Accuracy: {accuracy:.4f}")
    
    return voting_clf, test_predictions

# needs CV loop after testing

In [10]:
df_train = bucket_seasons(df_train)
df_test = bucket_seasons(df_test)

df_train = bucket_days(df_train)
df_test = bucket_days(df_test)

df_train['breed'] = df_train['breed'].astype(str)
df_test['breed'] = df_test['breed'].astype(str)

df_train = df_train.drop(columns=['size', 'intake_hour', 'intake_month'])
df_test = df_test.drop(columns=['size', 'intake_hour', 'intake_month'])

# breed is encoded, but dont fret
def encode_breed(val):
    if isinstance(val, int):
        return val  # Keep numeric as-is
    mapping = {'Common': 9, 'Rare': 10, 'Unknown': 11}
    return mapping.get(val, -1)  # Return -1 for unexpected values (or handle as needed)

# Apply transformation
df_train['breed'] = df_train['breed'].apply(encode_breed)
df_test['breed'] = df_test['breed'].apply(encode_breed)

df_train.head()

,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,intake_year,primary_color,is_mix,outcome_type,season,time_of_day
0,Stray,Normal / Behavior,Dog,Spayed Female,96.0,-1,2015,white,0,Return to Owner,Summer,Afternoon
1,Stray,Normal / Behavior,Dog,Intact Male,11.0,-1,2016,sable,1,Return to Owner,Spring,Evening
2,Public Assist,Normal / Behavior,Cat,Neutered Male,24.0,9,2022,orange,0,Transfer,Spring,Night
3,Owner Surrender,Normal / Behavior,Dog,Neutered Male,24.0,-1,2017,chocolate,1,Return to Owner,Winter,Afternoon
4,Public Assist,Normal / Behavior,Dog,Neutered Male,72.0,-1,2019,black,1,Return to Owner,Spring,Morning


In [11]:
X = df_train.drop(columns=['outcome_type'])
y = df_train['outcome_type']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=42)




In [12]:
cat_cols = ['intake_type', 'intake_condition', 'animal_type', 'sex_upon_intake', 'breed','season', 'is_mix', 'time_of_day', 'primary_color'] # experimenting with droping color
num_cols = ['age_upon_intake', 'intake_year']    # Replace with your actual numerical columns
# freq_cols = ['primary_color'] # for trees

'''
# This is just used to get the model with the best classifiers
models = [
    ('logreg', train_logreg_classifier(X_train, y_train, X_test, cat_cols, num_cols)[0]), #logistic regression
    ('rf', train_RF_classifier(X_train, y_train, X_test, cat_cols)[0]), #random forest
    ('xrt', train_XRT_classifier(X_train, y_train, X_test, cat_cols)[0]) #x random trees
]

# Assuming X_train, y_train, X_test are already defined in your environment
voting_model, test_predictions = train_voting_classifier(X_train, y_train, X_test, models, voting='soft')

classification_report_with_accuracy_score(y_test, test_predictions)
'''

"\n# This is just used to get the model with the best classifiers\nmodels = [\n    ('logreg', train_logreg_classifier(X_train, y_train, X_test, cat_cols, num_cols)[0]), #logistic regression\n    ('rf', train_RF_classifier(X_train, y_train, X_test, cat_cols)[0]), #random forest\n    ('xrt', train_XRT_classifier(X_train, y_train, X_test, cat_cols)[0]) #x random trees\n]\n\n# Assuming X_train, y_train, X_test are already defined in your environment\nvoting_model, test_predictions = train_voting_classifier(X_train, y_train, X_test, models, voting='soft')\n\nclassification_report_with_accuracy_score(y_test, test_predictions)\n"

In [13]:
X_train.head()

,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,intake_year,primary_color,is_mix,season,time_of_day
107615,Stray,Normal / Behavior,Cat,Intact Female,1.0,9,2021,torbie,0,Summer,Morning
13002,Stray,Injured,Dog,Intact Male,24.0,-1,2014,brown brindle,1,Summer,Morning
32513,Public Assist,Normal / Behavior,Dog,Intact Female,12.0,-1,2019,brown,1,Summer,Night
79497,Stray,Normal / Behavior,Cat,Intact Female,1.0,9,2015,calico,1,Summer,Evening
88489,Stray,Normal / Behavior,Dog,Intact Female,4.0,-1,2017,black,0,Spring,Morning


In [14]:


cat_cols = ['intake_type', 'intake_condition', 'animal_type', 'sex_upon_intake', 'breed','season', 'is_mix', 'time_of_day', 'primary_color'] # experimenting with droping color
num_cols = ['age_upon_intake', 'intake_year']    # Replace with your actual numerical columns
# freq_cols = ['primary_color'] # for trees
'''
# This is just used to get the model with the best classifiers
models = [
    ('logreg', train_logreg_classifier(X_train, y_train, X_test, cat_cols, num_cols)[0]), #logistic regression
    ('rf', train_RF_classifier(X_train, y_train, X_test, cat_cols)[0]), #random forest
    ('xrt', train_XRT_classifier(X_train, y_train, X_test, cat_cols)[0]) #x random trees
]

# Assuming X_train, y_train, X_test are already defined in your environment
voting_model, test_predictions = train_voting_classifier(X_train, y_train, X_test, models, voting='soft')

classification_report_with_accuracy_score(y_test, test_predictions)
'''

"\n# This is just used to get the model with the best classifiers\nmodels = [\n    ('logreg', train_logreg_classifier(X_train, y_train, X_test, cat_cols, num_cols)[0]), #logistic regression\n    ('rf', train_RF_classifier(X_train, y_train, X_test, cat_cols)[0]), #random forest\n    ('xrt', train_XRT_classifier(X_train, y_train, X_test, cat_cols)[0]) #x random trees\n]\n\n# Assuming X_train, y_train, X_test are already defined in your environment\nvoting_model, test_predictions = train_voting_classifier(X_train, y_train, X_test, models, voting='soft')\n\nclassification_report_with_accuracy_score(y_test, test_predictions)\n"

In [15]:
# assuming these are pipelines or models you've already trained/tuned
'''
voting_clf = VotingClassifier(
    estimators=[
        ('xgb', xgb_model),      # best XGBoost model
        ('cat', catboost_model), # best CatBoost model
        ('logreg', log_reg_model),
        ('rf', rf_model),
        ('nb', nb_model)
    ],
    voting='soft',
  #  weights=[3, 2.5, 2, 1.5, 1],  # optional: weight more accurate models
    n_jobs=-1
)

voting_clf.fit(X_train, y_train)
y_pred = voting_clf.predict(X_test)

# Evaluate
from sklearn.metrics import balanced_accuracy_score
print("Ensemble Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred))
'''


'\nvoting_clf = VotingClassifier(\n    estimators=[\n        (\'xgb\', xgb_model),      # best XGBoost model\n        (\'cat\', catboost_model), # best CatBoost model\n        (\'logreg\', log_reg_model),\n        (\'rf\', rf_model),\n        (\'nb\', nb_model)\n    ],\n    voting=\'soft\',\n  #  weights=[3, 2.5, 2, 1.5, 1],  # optional: weight more accurate models\n    n_jobs=-1\n)\n\nvoting_clf.fit(X_train, y_train)\ny_pred = voting_clf.predict(X_test)\n\n# Evaluate\nfrom sklearn.metrics import balanced_accuracy_score\nprint("Ensemble Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred))\n'

In [19]:
# used for extra valdiation to find the best thresholds to maximize recall per class
le = LabelEncoder()
y_train = le.fit_transform(y_train)

X_train_sub, X_val, y_train_sub, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

# Train on sub-training set
logreg_CV = train_logreg_classifier(X_train_sub, y_train_sub, X_val, cat_cols, num_cols)[0]
rf_CV = train_RF_classifier(X_train_sub, y_train_sub, X_val, cat_cols)[0]
xrt_CV = train_XRT_classifier(X_train_sub, y_train_sub, X_val, cat_cols)[0]

# Get predicted probabilities on validation set
log_probs_CV = logreg_CV.predict_proba(X_val)
rf_probs_CV = rf_CV.predict_proba(X_val)
xrt_probs_CV = xrt_CV.predict_proba(X_val)

# Average the probabilities
avg_probs_val = (log_probs_CV + rf_probs_CV + xrt_probs_CV) / 3


from itertools import product

# Define classes (adjust as needed)
classes = [0, 1, 2, 3, 4]

# Threshold options for each class
threshold_grid = {
    0: [0.5, 0.6, 0.7, 0.8],
    1: [0.05, 0.1, 0.2],
    2: [0.1, 0.2, 0.3],
    3: [0.2, 0.3, 0.4],
    4: [0.2, 0.3, 0.4]
}

# Generate all threshold combinations
all_thresholds = list(product(*threshold_grid.values()))

# Tune thresholds
best_score = -1
best_thresholds = None

for thresh_combo in all_thresholds:
    thresholds = dict(zip(classes, thresh_combo))
    preds = []

    for probs in avg_probs_val:
        passed = [cls for cls, prob in enumerate(probs) if prob >= thresholds.get(cls, 1.0)]
        if passed:
            chosen = max(passed, key=lambda cls: probs[cls])
        else:
            chosen = np.argmax(probs)
        preds.append(chosen)

    score = balanced_accuracy_score(y_val, preds)

    if score > best_score:
        best_score = score
        best_thresholds = thresholds

print("Best thresholds:", best_thresholds)
print("Best balanced accuracy on validation:", best_score)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END logreg__C=0.3845401188473625, logreg__class_weight={0: 0.404084067026413, 1: 21.299101796407186, 2: 6.461307901907357, 3: 1.3310693236037048, 4: 0.6360214573088958}, logreg__fit_intercept=True, logreg__max_iter=8000, logreg__penalty=l2, logreg__solver=saga;, score=0.467 total time=   1.6s
[CV 2/5] END logreg__C=0.3845401188473625, logreg__class_weight={0: 0.404084067026413, 1: 21.299101796407186, 2: 6.461307901907357, 3: 1.3310693236037048, 4: 0.6360214573088958}, logreg__fit_intercept=True, logreg__max_iter=8000, logreg__penalty=l2, logreg__solver=saga;, score=0.459 total time=   1.1s
[CV 3/5] END logreg__C=0.3845401188473625, logreg__class_weight={0: 0.404084067026413, 1: 21.299101796407186, 2: 6.461307901907357, 3: 1.3310693236037048, 4: 0.6360214573088958}, logreg__fit_intercept=True, logreg__max_iter=8000, logreg__penalty=l2, logreg__solver=saga;, score=0.469 total time=   0.9s


/u/nneoma/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 4/5] END logreg__C=0.3845401188473625, logreg__class_weight={0: 0.404084067026413, 1: 21.299101796407186, 2: 6.461307901907357, 3: 1.3310693236037048, 4: 0.6360214573088958}, logreg__fit_intercept=True, logreg__max_iter=8000, logreg__penalty=l2, logreg__solver=saga;, score=0.447 total time= 1.8min
[CV 5/5] END logreg__C=0.3845401188473625, logreg__class_weight={0: 0.404084067026413, 1: 21.299101796407186, 2: 6.461307901907357, 3: 1.3310693236037048, 4: 0.6360214573088958}, logreg__fit_intercept=True, logreg__max_iter=8000, logreg__penalty=l2, logreg__solver=saga;, score=0.468 total time=   1.8s
[CV 1/5] END logreg__C=0.6086584841970366, logreg__class_weight={0: 0.404084067026413, 1: 21.299101796407186, 2: 6.461307901907357, 3: 1.3310693236037048, 4: 0.6360214573088958}, logreg__fit_intercept=False, logreg__max_iter=8000, logreg__penalty=l2, logreg__solver=lbfgs;, score=0.467 total time=   1.7s
[CV 2/5] END logreg__C=0.6086584841970366, logreg__class_weight={0: 0.404084067026413, 1:

In [18]:
# manual voting
#def manual_voting(models):
logreg = train_logreg_classifier(X_train, y_train, X_test, cat_cols, num_cols)[0] #logistic regression
rf = train_RF_classifier(X_train, y_train, X_test, cat_cols)[0] #random forest
xrt = train_XRT_classifier(X_train, y_train, X_test, cat_cols)[0] #x random trees

log_probs = logreg.predict_proba(X_test)
rf_probs = rf.predict_proba(X_test)
xrt_probs = xrt.predict_proba(X_test)

avg_probs = (log_probs + rf_probs + xrt_probs) / 3


thresholds = {
    0: 0.60,  # Adopted
    1: 0.10,  # Died
    2: 0.20,  # Euth
    3: 0.30,  # RTO
    4: 0.30   # Transfer
}

le = LabelEncoder()
#le.fit_transform(y_train)

custom_preds = []

for probs in avg_probs:
    # Find which classes exceed their thresholds
    passed = [cls for cls, prob in enumerate(probs) if prob >= thresholds.get(cls, 1.0)]
    
    if passed:
        # Among the passed, pick the class with the highest probability
        chosen = max(passed, key=lambda cls: probs[cls])
    else:
        # Default to the class with highest average prob
        chosen = np.argmax(probs)
        
    custom_preds.append(chosen)

test_preds = np.array(custom_preds)
test_preds_labels = le.inverse_transform(custom_preds)

classification_report_with_accuracy_score(y_test, test_preds_labels)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


/u/nneoma/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 1/5] END logreg__C=0.3845401188473625, logreg__class_weight={'Adoption': 0.4040806125462931, 'Died': 21.29916167664671, 'Euthanasia': 6.4625, 'Return to Owner': 1.3310979717087044, 'Transfer': 0.6360118728319565}, logreg__fit_intercept=True, logreg__max_iter=8000, logreg__penalty=l2, logreg__solver=saga;, score=0.453 total time= 2.3min
[CV 2/5] END logreg__C=0.3845401188473625, logreg__class_weight={'Adoption': 0.4040806125462931, 'Died': 21.29916167664671, 'Euthanasia': 6.4625, 'Return to Owner': 1.3310979717087044, 'Transfer': 0.6360118728319565}, logreg__fit_intercept=True, logreg__max_iter=8000, logreg__penalty=l2, logreg__solver=saga;, score=0.456 total time=   1.8s
[CV 3/5] END logreg__C=0.3845401188473625, logreg__class_weight={'Adoption': 0.4040806125462931, 'Died': 21.29916167664671, 'Euthanasia': 6.4625, 'Return to Owner': 1.3310979717087044, 'Transfer': 0.6360118728319565}, logreg__fit_intercept=True, logreg__max_iter=8000, logreg__penalty=l2, logreg__solver=saga;, score

/u/nneoma/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/u/nneoma/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


[CV] END rf__max_depth=20, rf__min_samples_leaf=1, rf__min_samples_split=5, rf__n_estimators=100; total time=   3.4s
[CV] END rf__max_depth=20, rf__min_samples_leaf=1, rf__min_samples_split=5, rf__n_estimators=100; total time=   3.6s
[CV] END rf__max_depth=20, rf__min_samples_leaf=1, rf__min_samples_split=5, rf__n_estimators=100; total time=   3.3s
[CV] END rf__max_depth=20, rf__min_samples_leaf=1, rf__min_samples_split=5, rf__n_estimators=100; total time=   3.6s
[CV] END rf__max_depth=20, rf__min_samples_leaf=1, rf__min_samples_split=5, rf__n_estimators=100; total time=   2.2s
[CV] END rf__max_depth=10, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=100; total time=   1.7s
[CV] END rf__max_depth=10, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=100; total time=   1.0s
[CV] END rf__max_depth=10, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=100; total time=   1.0s
[CV] END rf__max_depth=10, rf__min_samples_leaf=1, rf__min_sampl

0.439206016338629

In [ ]:
'''
# Predict from each already-trained model
log_preds = logreg.predict(X_test)
rf_preds = rf.predict(X_test)
xrt_preds = xrt.predict(X_test)

# Combine predictions manually, e.g. majority vote
from scipy.stats import mode
all_preds = np.vstack([log_preds, rf_preds, xrt_preds])
final_preds, _ = mode(all_preds, axis=0)

'''